In [1]:
!antlr4 -Dlanguage=Python3 BraKet.g4 -visitor -no-listener

In [2]:
import sys
from antlr4 import *
from BraKetLexer import BraKetLexer
from BraKetParser import BraKetParser

def main(show_tokens=True, code=""):
    # 1. Provide the input
    input_stream = InputStream(code)

    # 2. Lexical Analysis
    lexer = BraKetLexer(input_stream)
    token_stream = CommonTokenStream(lexer)

    # 3. Parsing
    parser = BraKetParser(token_stream)
    tree = parser.program()  # Ensure 'prog' matches your .g4 start rule

    # 4. Print the Tree
    print("--- Parse Tree ---")
    print(tree.toStringTree(recog=parser))
    print()

    # 5. Output Tokens (Optional)
    if show_tokens:
        print("--- Tokens ---")
        # We fill the stream to ensure all tokens are loaded
        token_stream.fill()
        
        print(f"{'INDEX':<6} | {'TEXT':<10} | {'TYPE':<15} | {'LINE':<5} | {'COL':<5}")
        print("-" * 50)
        
        for i, token in enumerate(token_stream.tokens):
            # Map the integer type ID to the name you defined in the .g4 file
            t_type = BraKetParser.symbolicNames[token.type] if token.type != -1 else "EOF"
            
            print(f"{i:<6} | {token.text:<10} | {t_type:<15} | {token.line:<5} | {token.column:<5}")

if __name__ == '__main__':
    main()

--- Parse Tree ---
program

--- Tokens ---
INDEX  | TEXT       | TYPE            | LINE  | COL  
--------------------------------------------------
0      | <EOF>      | EOF             | 1     | 0    


In [3]:
code='''
from _mamamo_ import _puke,test, test
from test import test
import filename

const i = 0
const x = t+(-1+9i)
const arr = [x, i, 1+1i, 1+0i]
const |t> = (1,1i)
const <t0| = (0,1) @ (0,1)
const not_op = (|t>)
const op = ((1i, 2), (1i, 2))
const booltest = !1 > (2 < 2) || (3 && 0) == 1

func test_function () {}

func test2 () {}
'''

In [4]:
code='''
func testfunc() {}
func test2(x, y=1, x=2) {
if ( x == 0 ) {
test = 1
} elif ( x == 1 ) {
test = 2
} elif ( x == 2 ) {
test = 4
} else {
test = 0
} 
}
'''

In [5]:
code='''
func testfunc() {}
func test2(x) {
    test = 0
    for (i = 0; i < x; x = x + 1) {
        test = test + 1
    }
    test = 0
    while (test < 4) {
        test = test + 1
    }
    x = 0
    do {
        x = x + 1
        if (x == 1) {
            test = 1
        } elif (x == 2) {
            test = 2
        } else {
            test = 3
        }
    } while (x < 10)
}
main() {
    test2(2)
}
'''

In [6]:
code='''
const struct_test = {
 x=0,
 func test() {
     print(x)
 },
 y=1
}
func testFun(struct) {
    print(struct.x)
}
main() {
    struct_test2 = {
        x = 1,
        y = 1,
        func test() {
            print(y, x)
        }
    }
    testFun(struct_test2)
}
'''

In [7]:
main(code=code)

--- Parse Tree ---
(program (const_decl_list (const_decl const (var_decl struct_test = (expression (struct { (struct_value (var_decl x = (expression (num_expression (num_term (num_factor 0))))) , (struct_value (func_decl func test ( ) { (statement_list (statement (func_call_statement print ( (arg_list (arg x)) )))) }) , (struct_value (var_decl y = (expression (num_expression (num_term (num_factor 1)))))))) }))))) (func_decl_list (func_decl func testFun ( (param_list (identifier_list struct)) ) { (statement_list (statement (func_call_statement print ( (arg_list (arg (struct_access struct . x))) )))) })) (main_function main ( ) { (statement_list (statement (assign_statement (var_decl struct_test2 = (expression (struct { (struct_value (var_decl x = (expression (num_expression (num_term (num_factor 1))))) , (struct_value (var_decl y = (expression (num_expression (num_term (num_factor 1))))) , (struct_value (func_decl func test ( ) { (statement_list (statement (func_call_statement print ( (

In [16]:
code='''
const x = 1-1-1i
'''

In [17]:
main(code=code)

--- Parse Tree ---
(program (const_decl_list (const_decl const (var_decl x = (expression (num_expression (num_term (num_factor 1))))))))

--- Tokens ---
INDEX  | TEXT       | TYPE            | LINE  | COL  
--------------------------------------------------
0      | const      | CONST           | 2     | 0    
1      | x          | IDENTIFIER      | 2     | 6    
2      | =          | ASSIGN          | 2     | 8    
3      | 1          | INT             | 2     | 10   
4      | -1-1i      | COMPLEX         | 2     | 11   
5      | <EOF>      | EOF             | 3     | 0    
